In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import pickle
import time
import re
import pandas as pd


In [2]:

# def setup_driver(chrome_driver_path):
def setup_driver():
    # driver = webdriver.Chrome(executable_path=chrome_driver_path)
    # Setup Chrome driver

    chrome_options = webdriver.ChromeOptions()
    # chrome_options.add_argument('--headless')
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')
    driver = webdriver.Chrome(options=chrome_options)
    return driver


In [3]:

def login_sinta(driver, username, password):
    driver.get("https://sinta.kemdikbud.go.id/")
    time.sleep(3)
    username_field = driver.find_element(By.NAME, "username") if len(driver.find_elements(By.NAME, "username")) > 0 else driver.find_element(By.XPATH, '//input[@name="username"]')
    password_field = driver.find_element(By.NAME, "password") if len(driver.find_elements(By.NAME, "password")) > 0 else driver.find_element(By.XPATH, '//input[@name="password"]')
    
    username_field.send_keys(username)
    password_field.send_keys(password)
    
    print("Username field value: ", username_field.get_attribute("value"))
    print("Password field value: ", password_field.get_attribute("value"))
    
    login_button = driver.find_element(By.CSS_SELECTOR, 'button.btn.btn-block.btn-info')
    login_button.click()
    time.sleep(5)
    
    with open("sinta_session.pkl", "wb") as session_file:
        pickle.dump(driver.get_cookies(), session_file)
    
    print("Session saved successfully!")


In [5]:

def load_session(driver):
    with open("sinta_session.pkl", "rb") as session_file:
        cookies = pickle.load(session_file)
        for cookie in cookies:
            driver.add_cookie(cookie)
    driver.refresh()
    print("Session loaded successfully!")


In [7]:

def navigate_to_tab(driver, researcher_id, view):
    driver.get(f"https://sinta.kemdikbud.go.id/authors/profile/{researcher_id}/?view={view}")
    print(f"Navigated to {view} tab for researcher profile with ID: {researcher_id}")
    time.sleep(3)


In [8]:

def scrape_articles(driver):
    articles = []
    pagination_text_element = driver.find_element(By.CSS_SELECTOR, 'div.pagination-text')
    pagination_text = pagination_text_element.text
    match = re.search(r'Page \d+ of (\d+)', pagination_text)
    if match:
        total_pages = int(match.group(1))
        print(f"Total number of pages: {total_pages}")
    else:
        print("Unable to determine total number of pages.")
        total_pages = 1

    for page in range(1, total_pages + 1):
        print(f"Scraping page {page} of {total_pages}")
        time.sleep(3)
        div_elements = driver.find_elements(By.CSS_SELECTOR, 'div.ar-list-item.mb-5')
        for div in div_elements:
            try:
                title = div.find_element(By.CSS_SELECTOR, 'div.ar-title > a').text
            except:
                title = "N/A"
            try:
                journal = div.find_element(By.CSS_SELECTOR, 'a.ar-pub').text
            except:
                journal = "N/A"
            try:
                year = div.find_element(By.CSS_SELECTOR, 'a.ar-year').text
            except:
                year = "N/A"
            try:
                link = div.find_element(By.CSS_SELECTOR, 'div.ar-title > a').get_attribute("href")
            except:
                link = "N/A"
            
            articles.append({
                "Title": title,
                "Journal": journal,
                "Year": year,
                "Link": link
            })
        
        if page < total_pages:
            try:
                next_button = driver.find_element(By.LINK_TEXT, "Next")
                next_button.click()
            except:
                print("No next button found, stopping navigation.")
                break
    
    return articles

def save_articles_to_files(articles, filename_prefix):
    articles_df = pd.DataFrame(articles)
    articles_df.to_csv(f"{filename_prefix}_articles.csv", index=False)
    articles_df.to_excel(f"{filename_prefix}_articles.xlsx", index=False)
    print(articles_df)


In [9]:

def main():
    # chrome_driver_path = "/path/to/chromedriver"
    username = ""
    password = ""
    researcher_id = "5986966"
    
    # driver = setup_driver(chrome_driver_path)
    driver = setup_driver()
    try:
        login_sinta(driver, username, password)
        load_session(driver)
        
        # Scrape Scopus Articles
        navigate_to_tab(driver, researcher_id, "scopus")
        scopus_articles = scrape_articles(driver)
        save_articles_to_files(scopus_articles, "scopus")
        
        # Scrape WOS Articles
        navigate_to_tab(driver, researcher_id, "wos")
        wos_articles = scrape_articles(driver)
        save_articles_to_files(wos_articles, "wos")
        
    finally:
        driver.quit()

if __name__ == "__main__":
    main()


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//input[@name="username"]"}
  (Session info: chrome=130.0.6723.70); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF7B02C3AB5+28005]
	(No symbol) [0x00007FF7B02283B0]
	(No symbol) [0x00007FF7B00C580A]
	(No symbol) [0x00007FF7B0115A3E]
	(No symbol) [0x00007FF7B0115D2C]
	(No symbol) [0x00007FF7B015EA97]
	(No symbol) [0x00007FF7B013BA7F]
	(No symbol) [0x00007FF7B015B8B3]
	(No symbol) [0x00007FF7B013B7E3]
	(No symbol) [0x00007FF7B01075C8]
	(No symbol) [0x00007FF7B0108731]
	GetHandleVerifier [0x00007FF7B05B643D+3118829]
	GetHandleVerifier [0x00007FF7B0606C90+3448640]
	GetHandleVerifier [0x00007FF7B05FCF0D+3408317]
	GetHandleVerifier [0x00007FF7B038A40B+841403]
	(No symbol) [0x00007FF7B023340F]
	(No symbol) [0x00007FF7B022F484]
	(No symbol) [0x00007FF7